### Prediction Model

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64
SEQUENCE_LENGTH = 20
CLASSES_LIST = ["smoke", "shoot_gun", "run", "hit"]
convlstm_model_h5 = load_model("./convlstm_model.h5", compile=False)
convlstm_model_h5.compile(
    optimizer="rmsprop",
    loss=None,
    metrics=None,
    loss_weights=None,
    weighted_metrics=None,
    run_eagerly=None,
    steps_per_execution=None,
    jit_compile=None,
)


def predict_single_action(video_file_path, SEQUENCE_LENGTH):
    video_reader = cv2.VideoCapture(video_file_path)
    frames_list = []
    predicted_class_name = ""
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(video_frames_count / SEQUENCE_LENGTH), 1)
    for frame_counter in range(SEQUENCE_LENGTH):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        success, frame = video_reader.read()
        if not success:
            break
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        normalized_frame = resized_frame / 255
        frames_list.append(normalized_frame)
    predicted_labels_probabilities = convlstm_model_h5.predict(
        np.expand_dims(frames_list, axis=0)
    )[0]
    predicted_label = np.argmax(predicted_labels_probabilities)
    predicted_class_name = CLASSES_LIST[predicted_label]
    print(
        f"Action Predicted: {predicted_class_name}\nConfidence: {predicted_labels_probabilities[predicted_label]}"
    )
    video_reader.release()


# for i in range(12):
#     input_file = f"./video_clips/test_Trim{i+1}.mp4"
#     print(f"{i}. {input_file}\n")
#     predict_single_action(input_file, SEQUENCE_LENGTH)

In [2]:
predict_single_action("C:/Users/toufiqhussain/Downloads/Untitled video - Made with Clipchamp.mp4", 20)

1/1 [==============================] - 1s 657ms/step
Action Predicted: smoke
Confidence: 0.9886788725852966


### Prediction Engine

In [ ]:
import cv2
import os
import shutil
import threading

def prediction_engine():
    while True:
        dir = os.listdir("D:/pycharm/simple_deep_sort/live_footages")
        for i in dir:
            if i and i.startswith("f"):
                path = os.path.join(
                    "D:/pycharm/simple_deep_sort/live_footages/", i
                )
                print(path)
                predict_single_action(path, 20)
                shutil.move(path, "D:/pycharm/simple_deep_sort/upload_engine")

def main():
    thread_1 = threading.Thread(target=prediction_engine)
    thread_1.start()
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open camera.")
        return

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(5))

    fourcc = cv2.VideoWriter_fourcc(*"H264")  # Codec for MP4 format
    stream_1 = None
    stream_2 = None
    s1_fileName = None
    s2_fileName = None
    recording_timer = 0
    file_no = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Error: Could not read frame.")
            if len(os.listdir("D:/pycharm/simple_deep_sort/live_footages"))>0:
                for i in os.listdir("D:/pycharm/simple_deep_sort/live_footages"):
                    os.rename(
                    f"D:/pycharm/simple_deep_sort/live_footages/{i}",
                    f"D:/pycharm/simple_deep_sort/live_footages/f_{i}",
                )
            thread_1.join()
            break
        if recording_timer == fps * 0:
            stream_1 = cv2.VideoWriter(
                "D:/pycharm/simple_deep_sort/live_footages/output_{}_s1.mp4".format(
                    file_no
                ),
                fourcc,
                fps,
                (frame_width, frame_height),
            )

            s1_fileName = "output_{}_s1.mp4".format(file_no)
            file_no += 1
        if recording_timer == fps * 7:
            stream_2 = cv2.VideoWriter(
                "D:/pycharm/simple_deep_sort/live_footages/output_{}_s2.mp4".format(
                    file_no
                ),
                fourcc,
                fps,
                (frame_width, frame_height),
            )
            s2_fileName = "output_{}_s2.mp4".format(file_no)
            file_no += 1

        if recording_timer >= fps * 0 and recording_timer <= fps * 10:
            stream_1.write(frame)
            if recording_timer == fps * 10:
                stream_1.release()
                os.rename(
                    f"D:/pycharm/simple_deep_sort/live_footages/{s1_fileName}",
                    f"D:/pycharm/simple_deep_sort/live_footages/f_{s1_fileName}",
                )
        if recording_timer >= fps * 7 and recording_timer <= fps * 17:
            stream_2.write(frame)
            if recording_timer == fps * 17:
                stream_2.release()
                os.rename(
                    f"D:/pycharm/simple_deep_sort/live_footages/{s2_fileName}",
                    f"D:/pycharm/simple_deep_sort/live_footages/f_{s2_fileName}",
                )

        if recording_timer == fps * 17:
            recording_timer = 0
        else:
            recording_timer += 1

        cv2.imshow("Recording", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    if stream_1 is not None:
        stream_1.release()
    if stream_2 is not None:
        stream_2.release()

    cap.release()
    cv2.destroyAllWindows()
    thread_1.join()


if __name__ == "__main__":
    main()